In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth

In [ ]:
!pip install vllm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 1

In [ ]:
import torch
import os

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available, using CPU instead")

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

GPU is available


In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)

True
12.6


In [ ]:
from unsloth import FastLanguageModel  # FastVisionModel for LLMs
import torch
max_seq_length = 2048  # Choose any! We auto support RoPE Scaling internally!
load_in_4bit = True  # Use 4bit quantization to reduce memory usage. Can be False.

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 07-31 06:25:21 [__init__.py:235] Automatically detected platform cuda.


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-4-unsloth-bnb-4bit",  # Phi-4 Unsloth Dynamic 4-bit Quant
    max_seq_length = max_seq_length,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.7.11: Fast Llama patching. Transformers: 4.54.0. vLLM: 0.10.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.39G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

We now add LoRA adapters for parameter efficient finetuning - this allows us to only efficiently train 1% of all parameters.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.7.11 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


We use our get_chat_template function to get the correct chat template

In [ ]:
#import the dataset , im using custom dataset
import json

file = json.load(open("json_extraction_dataset_500.json", "r"))

print(file[5])

{'input': "Extract the product information:\n<div class='product'><h2>Razer DeathAdder V2</h2><span class='price'>$1277</span><span class='category'>audio</span><span class='brand'>Apple</span></div>", 'output': {'name': 'Razer DeathAdder V2', 'price': '$1277', 'category': 'audio', 'manufacturer': 'Apple'}}


In [ ]:
print(file[5]['input'])

Extract the product information:
<div class='product'><h2>Razer DeathAdder V2</h2><span class='price'>$1277</span><span class='category'>audio</span><span class='brand'>Apple</span></div>


In [ ]:
print(file[5]['output'])

{'name': 'Razer DeathAdder V2', 'price': '$1277', 'category': 'audio', 'manufacturer': 'Apple'}


In [ ]:
from datasets import Dataset

dataset = Dataset.from_list(file)

In [ ]:
dataset[0]

{'input': "Extract the product information:\n<div class='product'><h2>Asus ROG Strix</h2><span class='price'>$1106</span><span class='category'>electronics</span><span class='brand'>Amazon</span></div>",
 'output': {'category': 'electronics',
  'manufacturer': 'Amazon',
  'name': 'Asus ROG Strix',
  'price': '$1106'}}

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-4",
)

# Format output dictionary to string for assistant message
def dict_to_str(d):
    return "\n".join([f"{k}: {v}" for k, v in d.items()])

def formatting_prompts_func(examples):
    conversations = []
    for input_text, output_dict in zip(examples["input"], examples["output"]):
        convo = [
            {"role": "user", "content": input_text},
            {"role": "assistant", "content": dict_to_str(output_dict)},
        ]
        conversations.append(tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False))

    return {"text": conversations}

In [ ]:
# Apply the formatting
formatted_dataset = dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
# Preview
print(formatted_dataset[0]["text"])

<|im_start|>user<|im_sep|>Extract the product information:
<div class='product'><h2>Asus ROG Strix</h2><span class='price'>$1106</span><span class='category'>electronics</span><span class='brand'>Amazon</span></div><|im_end|><|im_start|>assistant<|im_sep|>category: electronics
manufacturer: Amazon
name: Asus ROG Strix
price: $1106<|im_end|>


In [ ]:
formatted_dataset

Dataset({
    features: ['input', 'output', 'text'],
    num_rows: 500
})

In [ ]:
print(dataset[5]['input'],dataset[5]['output'])

Extract the product information:
<div class='product'><h2>Razer DeathAdder V2</h2><span class='price'>$1277</span><span class='category'>audio</span><span class='brand'>Apple</span></div> {'category': 'audio', 'manufacturer': 'Apple', 'name': 'Razer DeathAdder V2', 'price': '$1277'}


In [ ]:
formatted_dataset[5]['text']

"<|im_start|>user<|im_sep|>Extract the product information:\n<div class='product'><h2>Razer DeathAdder V2</h2><span class='price'>$1277</span><span class='category'>audio</span><span class='brand'>Apple</span></div><|im_end|><|im_start|>assistant<|im_sep|>category: audio\nmanufacturer: Apple\nname: Razer DeathAdder V2\nprice: $1277<|im_end|>"

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"  # or args = TrainingArguments(report_to="none", ...)

In [ ]:
from trl import SFTConfig, SFTTrainer
from transformers import DataCollatorForSeq2Seq
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_dataset, # Use the formatted dataset
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    packing = False, # Can make training 5x faster for short sequences.
    # Removed formatting_func since we are using the formatted_dataset
    args = SFTConfig(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 20,
        learning_rate = 2e-4,
        logging_steps = 2,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part="<|im_start|>user<|im_sep|>",
    response_part="<|im_start|>assistant<|im_sep|>",
)

Map (num_proc=2):   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

"<|im_start|>user<|im_sep|>Extract the product information:\n<div class='product'><h2>Razer DeathAdder V2</h2><span class='price'>$1277</span><span class='category'>audio</span><span class='brand'>Apple</span></div><|im_end|><|im_start|>assistant<|im_sep|>category: audio\nmanufacturer: Apple\nname: Razer DeathAdder V2\nprice: $1277<|im_end|>"

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                            category: audio\nmanufacturer: Apple\nname: Razer DeathAdder V2\nprice: $1277<|im_end|>'

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 1 | Total steps = 20
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 65,536,000 of 14,725,043,200 (0.45% trained)


Step,Training Loss
2,2.521900
4,2.216200
6,1.134600


Step,Training Loss
2,2.521900
4,2.216200
6,1.134600
8,0.312200
10,0.044400
12,0.007300
14,0.001800
16,0.000800
18,0.000400
20,0.000300


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-4",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {
        "role": "user",
        "content": "Extract the product information:\n<div class='product'><h2>Logitech MX Master 3</h2><span class='price'>$258</span><span class='category'>cameras</span><span class='brand'>Amazon</span></div>",
    }
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(
    input_ids = inputs, max_new_tokens = 64, use_cache = True, temperature = 1.5, min_p = 0.1
)
resp = tokenizer.batch_decode(outputs)

In [ ]:
text = resp[0].split("assistant")[1].replace("<|im_sep|>", "").replace("<|im_end|>", "").strip()

'category: cameras\nmanufacturer: Amazon\nname: Logitech MX Master 3\nprice: $258'

In [ ]:
from IPython.display import Markdown, display

# Redefine text
text = resp[0].split("assistant")[1].replace("<|im_sep|>", "").replace("<|im_end|>", "").strip()

display(Markdown(text))

category: cameras
manufacturer: Amazon
name: Logitech MX Master 3
price: $258

In [ ]:
import json

def text_to_json(text):
    # Split by lines and parse key-value pairs
    data = dict(line.split(": ", 1) for line in text.strip().splitlines())
    return json.dumps(data, indent=2)

# Convert and print JSON
json_output = text_to_json(text)
print(json_output)

{
  "category": "cameras",
  "manufacturer": "Amazon",
  "name": "Logitech MX Master 3",
  "price": "$258"
}
